# Talktorial 11 (part C)

# CADD web services that can be used via a Python API

__Developed at AG Volkamer, Charité__

Dr. Jaime Rodríguez-Guerra

## Aim of this talktorial

> This is part C of the "Online webservices" talktorial:
>
> - 11a. Querying KLIFS & PubChem for potential kinase inhibitors
> - 11b. Docking the candidates against the target obtained in 11a
> - __11c. Assessing the results and comparing against known data__

After obtaining input structures and docking them, we will assess whether the results are any good.

## Learning goals

### Theory

- Protein-ligand interactions
- False positives in docking

### Practical

- Visualize the results
- Run automated analysis

### Discussion

Pending.

### Quiz

Pending.

***

# Theory

## Protein-ligand interactions

Pending

## False positives

Pending

***

# Practical

## Visualize the results

Use `nglview` for that! It's compatible with `PDBQT` files out of the box.

In [78]:
import nglview as nv
from glob import glob

In [89]:
def split_pdbqt(path):
    """
    Split a multimodel PDBQT into separate files.
    """
    with open(path) as f:
        lines = []
        i = 0
        for line in f:
            lines.append(line)
            if line.strip() == 'ENDMDL':
                with open(f'data/results.{i}.pdbqt', 'w') as o:
                    o.write(''.join(lines))
                i += 1
                lines = []

In [87]:
def show_docking_results(protein, *ligands):
    v = nv.show_file(protein)
    for ligand in ligands:
        v.add_component(ligand)
    return v

In [91]:
!tail -13 data/vina.out

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1         -7.2      0.000      0.000
   2         -7.0      4.868      6.604
   3         -6.9      1.803      3.295
   4         -6.6      1.714      2.128
   5         -6.5      1.451      1.569
   6         -6.5      1.663      3.057
   7         -6.5      1.428      2.623
   8         -6.4      4.968      6.810
   9         -6.3      1.801      2.936
Writing output ... done.


In [88]:
split_pdbqt("data/results.pdbqt")
v = show_docking_results("data/protein.mol2", *list(glob("data/results.*.pdbqt")))
v

NGLWidget()